In [1]:
import urllib.request
from bs4 import BeautifulSoup
import re as re
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
import time
import click

In [38]:
#All information is gathered from www.weather.com

headers = {
    'User-Agent': 'Weather bot',
    'From': 'dylansuther12@gmail.com'
}


#Page information for selenium
baseURL = 'https://www.weather.com'

#Chromedriver information for selenium
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver.get(baseURL)

def menu():
    welcome = "Hi! Please choose weather data you would like to see!" 
    menuChoices = "\n1. Today's Forecast\n2. Ten Day Forecast\n3. Enter a new location\n4. Exit\n"
    choice = input(welcome + "\n" + "-" * len(welcome) + menuChoices)
    return choice

def chooseAgain():
    contChoice = input(contString)
    if(contChoice == 'y' or contChoice == 'Y'):
        return True
    elif(contChoice == 'n' or contChoice == 'N'):
        print("Thanks! Have a wonderful day!")
        return False
    else:
        print("Invalid option. Please select again")
        return chooseAgain()
    
    
    

#Search for address using selenium
def searchWeather(location):
    try:
        address = driver.find_element_by_id("LocationSearch_input")
        time.sleep(2)
        address.send_keys(location)
        time.sleep(2)
        address.send_keys(Keys.ENTER)

    except StaleElementReferenceException:
        address = driver.find_element_by_id("LocationSearch_input")
        time.sleep(2)
        address.send_keys(location)
        time.sleep(2)
        address.send_keys(Keys.ENTER)
        
#Print out weather forecast for today
def todaysWeather():
    
    soupURL = driver.current_url
    page = urllib.request.urlopen(soupURL).read()
    weatherSoup = BeautifulSoup(page)
    
    weatherData = {
        'location' : weatherSoup.find(class_ = 'CurrentConditions--location--1YWj_').text,
        'currentTemp': weatherSoup.find(class_ = "CurrentConditions--tempValue--MHmYY").text,
        'windSpeed' : weatherSoup.find(class_ = "Wind--windWrapper--3Ly7c undefined").text[14:],
        'currentConditions': weatherSoup.find(class_ = "CurrentConditions--phraseValue--mZC_p").text
    }
    
    print("\nHere is today's weather forecast for " + weatherData['location'] + "\n" + "-" * + len("Here is today's weather data for " +
                                                                               weatherData['location']))
    print("Current Temperature: " + weatherData['currentTemp'] + "\nCurrent Wind Speed: " + 
          weatherData['windSpeed'] + "\nCurrent Weather Conditions: " + weatherData['currentConditions'])

#Print out ten day weather forecast
def tendayWeather():
    soupURL = driver.current_url[:27] + "/tenday" + driver.current_url[33:]
    page = urllib.request.urlopen(soupURL).read()
    weatherSoup = BeautifulSoup(page)
    
    detailIndex = weatherSoup.find_all(id = re.compile("detailIndex"))
    
    location = weatherSoup.find(class_ = "LocationPageTitle--PresentationName--1AMA6").text
    
    print("\nHere is the ten day weather forecast for " + location + "\n" + "-" * + len("Here is today's weather data for " +
                                                                               location))


    for i in range(0, 10):
        weatherData = {
            'date' : detailIndex[i].find(class_ = "DailyContent--daypartDate--3VGlz").text,
            'highTemp' : detailIndex[i].find(class_ = "DetailsSummary--highTempValue--3PjlX").text,
            'lowTemp' : detailIndex[i].find(class_ = "DetailsSummary--lowTempValue--2tesQ").text,
            'precipitation' : detailIndex[i].find(class_ = "DailyContent--value--1Jers").text,
            'windSpeed' : detailIndex[i].find(class_ = "Wind--windWrapper--3Ly7c DailyContent--value--1Jers").text.split(" ")[1] + " mph",
            'summary' : detailIndex[i].find(class_="DailyContent--narrative--3Ti6_").text
        }


        #weatherDataList.append(weatherData)
        weatherString = ("\n"
        + click.style(weatherData['date'], bold=True) 
        + click.style("\n\nHigh Temperature: ", bold=True) 
        + weatherData['highTemp'] 
        + click.style(" F\nLow Temperature: ", bold=True) 
        + weatherData['lowTemp'] 
        + click.style(" F\nPrecipitation: ", bold=True) 
        + weatherData['precipitation'] 
        + click.style("\nWind Speed: ", bold=True) 
        + weatherData['windSpeed'] 
        + "\n\n")  

        box = "-" * (len(max(weatherString.split("\n"))) + 4)

        print(f"{box}\n{weatherString.center(len(box)-2)}\n{box}")
       
    
    
    
    
location = input("Enter a location to get weather information: ")
searchWeather(location)

menuFlag = True
contString = "Would you like to make another choice? (y/n)"

while(menuFlag):
    menuChoice = menu()
    if(menuChoice == '1'):
        todaysWeather()
        menuFlag = chooseAgain()
    elif(menuChoice == '2'):
        tendayWeather()
        menuFlag = chooseAgain()
    elif(menuChoice == '3'):
        location = input("Enter a location to get weather information: ")
        searchWeather(location)
    elif(menuChoice == '4'):
        print("Thanks! Have a wonderful day!")
        menuFlag = False
    else:
        print("Invalid option. Please select again")
        
    




Enter a location to get weather information: Bowling Green OH
Hi! Please choose weather data you would like to see!
-----------------------------------------------------
1. Today's Forecast
2. Ten Day Forecast
3. Enter a new location
4. Exit
1

Here is today's weather forecast for Bowling Green, OH
--------------------------------------------------
Current Temperature: 48°
Current Wind Speed: 14 mph
Current Weather Conditions: Rain Shower
Would you like to make another choice? (y/n)fewg
Invalid option. Please select again
Would you like to make another choice? (y/n)wegweg
Invalid option. Please select again
Would you like to make another choice? (y/n)hrehra
Invalid option. Please select again
Would you like to make another choice? (y/n)y
Hi! Please choose weather data you would like to see!
-----------------------------------------------------
1. Today's Forecast
2. Ten Day Forecast
3. Enter a new location
4. Exit
2

Here is the ten day weather forecast for Bowling Green, OH
----------